## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Severo-Kurilsk,RU,50.6789,156.1250,52.54,93,84,7.61,broken clouds
1,1,Williston,US,48.1470,-103.6180,90.34,33,63,1.99,broken clouds
2,2,Bluff,NZ,-46.6000,168.3333,37.80,74,100,16.31,overcast clouds
3,3,Avarua,CK,-21.2078,-159.7750,77.05,78,40,18.41,scattered clouds
4,4,Yellowknife,CA,62.4560,-114.3525,63.12,59,75,10.36,light intensity shower rain


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Williston,US,48.1470,-103.6180,90.34,33,63,1.99,broken clouds
3,3,Avarua,CK,-21.2078,-159.7750,77.05,78,40,18.41,scattered clouds
5,5,Cabo San Lucas,MX,22.8909,-109.9124,94.60,43,40,13.80,scattered clouds
7,7,Luwuk,ID,-0.9516,122.7875,75.47,91,82,3.78,broken clouds
8,8,Georgetown,MY,5.4112,100.3354,82.33,93,20,4.61,few clouds
11,11,Salalah,OM,17.0151,54.0924,77.09,88,75,4.61,broken clouds
13,13,Waingapu,ID,-9.6567,120.2641,71.64,88,10,6.78,clear sky
14,14,Hithadhoo,MV,-0.6000,73.0833,81.57,74,100,16.60,light rain
20,20,Richards Bay,ZA,-28.7830,32.0377,66.04,76,100,8.72,light rain
21,21,Zhanaozen,KZ,43.3000,52.8000,82.47,48,0,13.42,clear sky


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                428
City                   428
Country                428
Lat                    428
Lng                    428
Max Temp               428
Humidity               428
Cloudiness             428
Wind Speed             428
Current Description    428
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Williston,US,48.1470,-103.6180,90.34,33,63,1.99,broken clouds
3,3,Avarua,CK,-21.2078,-159.7750,77.05,78,40,18.41,scattered clouds
5,5,Cabo San Lucas,MX,22.8909,-109.9124,94.60,43,40,13.80,scattered clouds
7,7,Luwuk,ID,-0.9516,122.7875,75.47,91,82,3.78,broken clouds
8,8,Georgetown,MY,5.4112,100.3354,82.33,93,20,4.61,few clouds
11,11,Salalah,OM,17.0151,54.0924,77.09,88,75,4.61,broken clouds
13,13,Waingapu,ID,-9.6567,120.2641,71.64,88,10,6.78,clear sky
14,14,Hithadhoo,MV,-0.6000,73.0833,81.57,74,100,16.60,light rain
20,20,Richards Bay,ZA,-28.7830,32.0377,66.04,76,100,8.72,light rain
21,21,Zhanaozen,KZ,43.3000,52.8000,82.47,48,0,13.42,clear sky


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Williston,US,90.34,broken clouds,48.1470,-103.6180,
3,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
5,Cabo San Lucas,MX,94.60,scattered clouds,22.8909,-109.9124,
7,Luwuk,ID,75.47,broken clouds,-0.9516,122.7875,
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
11,Salalah,OM,77.09,broken clouds,17.0151,54.0924,
13,Waingapu,ID,71.64,clear sky,-9.6567,120.2641,
14,Hithadhoo,MV,81.57,light rain,-0.6000,73.0833,
20,Richards Bay,ZA,66.04,light rain,-28.7830,32.0377,
21,Zhanaozen,KZ,82.47,clear sky,43.3000,52.8000,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Williston,US,90.34,broken clouds,48.1470,-103.6180,Hampton Inn & Suites Williston
3,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
5,Cabo San Lucas,MX,94.60,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
7,Luwuk,ID,75.47,broken clouds,-0.9516,122.7875,Hotel Santika Luwuk
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
11,Salalah,OM,77.09,broken clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
13,Waingapu,ID,71.64,clear sky,-9.6567,120.2641,Pondok Wisata Elim
14,Hithadhoo,MV,81.57,light rain,-0.6000,73.0833,Scoop Guest House
20,Richards Bay,ZA,66.04,light rain,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
21,Zhanaozen,KZ,82.47,clear sky,43.3000,52.8000,K.korabay 28A


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))